In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow, get_cmap
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 24
TITLE = "24-25 March 2023"
SUBTITLE = "12 PM 25 March 2023"
SETPOINT_LOCS = {}
sts = utc(2023, 3, 25, 6)
ets = utc(2023, 3, 26, 20)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
df = df[~df['nwsli'].isin(['DSXI4', 'DMX'])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df[df["state"] == 'IA'])
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(
        lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df[df["state"] == 'IA'])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [9]:
def main():
    setpoints = [
        (39, 0),
        (55, 0),
        (86, 0),
        (23, 0),
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [

    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
    # draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='230325.png')
    mp.close()

main()

     state  wfo   val      lon      lat                             geo  \
24      IA  DVN  11.3 -90.5400  41.6900  POINT (783685.035 -322815.383)   
36      IA  DVN  10.2 -90.7300  41.8400  POINT (766204.135 -308013.223)   
38      IA  DVN  10.0 -90.9200  41.8600  POINT (750341.807 -307556.418)   
42      IA  DVN   9.5 -90.4000  42.0500  POINT (790675.449 -281645.383)   
43      IA  DVN   9.5 -90.6700  41.5200  POINT (775050.705 -342858.182)   
...    ...  ...   ...      ...      ...                             ...   
1775    IA  OAX   0.0 -95.3906  41.3133  POINT (384925.117 -399093.372)   
1776    IA  OAX   0.0 -95.5667  40.8667  POINT (372813.096 -449482.336)   
1777    IA  OAX   0.0 -95.2333  41.0000  POINT (399987.787 -433122.842)   
1778    IA  OAX   0.0 -95.3500  40.7833  POINT (391515.463 -457736.398)   
1779    IA  OAX   0.0 -95.6811  41.3894  POINT (360265.235 -391959.396)   

     used_for_analysis  nwsli plotme source  xcell  ycell  
24                True     24   True   

/opt/miniconda3/envs/prod/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
